In [ ]:
from src.textclf_transformer import *
import pandas as pd
from datasets import load_dataset, concatenate_datasets
from sklearn.model_selection import train_test_split

# Tokenizacja danych

In [ ]:
ds = load_dataset("imdb")
merged = concatenate_datasets([ds["test"], ds["train"]])
df = pd.DataFrame(merged)


In [ ]:
tokenizer = WordPieceTokenizerWrapper()
tokenizer.load("src/textclf_transformer/tokenizer/BERT_original")

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=42, stratify=test_df['label'])

print(f"Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")

In [ ]:
tokens_train = tokenizer.encode_pandas(train_df, 'text',
                                  max_length=512, 
                                  label_col='label')
tokens_val = tokenizer.encode_pandas(val_df, 'text',
                                max_length=512,
                                label_col='label')
tokens_test = tokenizer.encode_pandas(test_df, 'text',
                                 max_length=512,
                                 label_col='label')

In [ ]:
save_dir = Path("./data/tokenized")

torch.save(tokens_train, save_dir / "imdb_train.pt")
torch.save(tokens_val, save_dir / "imdb_val.pt")
torch.save(tokens_test, save_dir / "imdb_test.pt")

# Generacja eksperymentu - finetuning

Utworzenie eksperymentu finetuning na zbiorze imdb na podstawie wcześniej wykonanego pretreningu TAPT

In [ ]:
!python experiments/generate_finetuning_experiment.py -f imdb_demo_pz -p E2_pretraining_imdb_bertsmall_favor_nb0.125

# Uruchomienie finetuningu

In [ ]:
!python train.py -n imdb_demo_pz -m finetuning